# Урок 9: White-box Knowledge Distillation

Самый плотный перенос знаний через доступ к логитам (вероятностям) и скрытым состояниям Учителя.

## 1. Математическая теория

### 1.1. Logit Distillation
Мы учим Студента предсказывать не только правильное слово, но и «почти правильные» слова (распределение вероятностей).
$$\mathcal{L}_{KD} = \text{KL}(P_{Teacher}^\tau || Q_{Student}^\tau)$$
Где $\tau$ - температура, увеличивающая «мягкость» распределения.

### 1.2. Методы из обзора:
*   **MiniLLM (Gu et al., 2024):** Предлагает использовать **Reverse KL**. Это заставляет маленького Студента фокусироваться на самых вероятных модах Учителя, предотвращая «размазывание» знаний, которое ведет к галлюцинациям.
*   **GKD (Agarwal et al., 2024):** Использует дистилляцию на «собственных» ошибках студента (On-policy), что резко повышает стабильность генерации длинных текстов.
*   **TED (Liang et al., 2023):** Метод «Явной дистилляции», при котором Студент обучается точно копировать активации конкретных слоев Учителя, выбранных на основе их важности для задачи.

---

In [ ]:
import torch.nn.functional as F

def calculate_kl_raw(s_logits, t_logits, tau=2.0):
    p = F.softmax(t_logits / tau, dim=-1)
    log_q = F.log_softmax(s_logits / tau, dim=-1)
    return F.kl_div(log_q, p, reduction='batchmean') * (tau**2)

print("nanoGPT Track: Реализована математика KL-дивергенции с температурой.")

## 2. Промышленная реализация: Custom KDTrainer
В индустрии KD интегрируется в стандартные Trainer'ы через переопределение метода `compute_loss`.

In [ ]:
"""
from transformers import Trainer
class KDTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        s_out = model(**inputs)
        with torch.no_grad(): t_out = self.teacher(**inputs)
        loss = s_out.loss + calculate_kl_raw(s_out.logits, t_out.logits)
        return (loss, s_out) if return_outputs else loss
"""
print("Llama Track: White-box дистилляция через кастомные функции потерь.")